# Interagindo com Neo4j usando Cypher

## Conectando ao banco de dados e limpando a base (ignorar)

In [2]:
import sys; sys.path.insert(0, '../../resources/python/')
from datasci4health.graphdrawer.visgraph import draw

In [3]:
from neo4j import GraphDatabase

URI = "neo4j://localhost"
with GraphDatabase.driver(URI) as driver:
    driver.verify_connectivity()

<ipython-input-3-548a69b4459e>:5: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


In [4]:
from py2neo import Graph

graph = Graph("bolt://127.0.0.1:7687")
graph.delete_all()

print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.relationships)))

Number of nodes: 0 
Number of relationships: 0 


## Usando uma extensão para fazer consultas Cypher 

Documentação:
https://ipython-cypher.readthedocs.io/en/latest/introduction.html


In [5]:
!python -m graph_notebook.ipython_profile.configure_ipython_profile

In [6]:
%load_ext graph_notebook.magics

The graph_notebook.magics extension is already loaded. To reload it, use:
  %reload_ext graph_notebook.magics


In [15]:
%%graph_notebook_config
{
  "host": "0.0.0.0",
  "port": 7687,
  "ssl": false,
  "neo4j": {
    "username": "neo4j",
    "password": "password",
    "auth": false,
    "database": ""
  }
}

set notebook config to:
{
  "host": "localhost",
  "port": 7687,
  "proxy_host": "",
  "proxy_port": 8182,
  "ssl": false,
  "sparql": {
    "path": ""
  },
  "gremlin": {
    "traversal_source": "g"
  }
}


In [16]:
%%opencypher
CREATE (person1:Person {name:'Asdrubal'}),
       (person2:Person {name:'Marcos'}   )
return person1, person2

### Criando nós e visualizando resultado

In [5]:
%cypher CREATE (node1:TipoA {propriedade1:'valor1', propriedade2:'valor2', label:'meu primeiro nó'})       

draw(graph, {}, physics=True)

1 nodes created.
3 properties set.
1 labels added.


In [6]:
%cypher CREATE (node1:Pessoa {nome:'Fulano Silva', profissao:'Professor', label:'Prof. Fulano'})       

draw(graph, {}, physics=True)

1 nodes created.
3 properties set.
1 labels added.


In [7]:
%cypher CREATE (node1:Pessoa {nome:'Cicrano Junior', profissao:'Engenheiro de dados', label:'Eng. Cicrano'})       

draw(graph, {}, physics=True)

1 nodes created.
3 properties set.
1 labels added.


### Criando relacionamentos (Arestas)

In [8]:
%cypher CREATE (person1:Pessoa {nome:'Matheus Mota', label:'Matheus'})-[:DEU_LIKE]->(movie1:Filme {titulo:'Django', ano:2009, label:'Django'})

draw(graph, {}, physics=True)

2 nodes created.
5 properties set.
1 relationships created.
2 labels added.


In [9]:
%cypher CREATE (person1:Pessoa {nome:'Laura Lero', label:'Laura'})-[:DEU_DESLIKE]->(movie1:Filme {titulo:'The Matrix', ano:1999, label:'Matrix'})

draw(graph, {}, physics=True)

2 nodes created.
5 properties set.
1 relationships created.
2 labels added.


### Ligando dois nós já existentes

In [10]:
%%cypher
MATCH (node_pessoa:Pessoa), (node_filme:Filme)
WHERE  node_pessoa.nome = 'Laura Lero' AND  node_filme.titulo = 'Django'
CREATE (node_pessoa)-[:DEU_LIKE]->(node_filme)


1 relationships created.


[]

In [11]:
draw(graph, {}, physics=True)

### Querying

In [12]:
%%cypher
MATCH (node_pessoa:Pessoa)-[:DEU_LIKE]->(node_filme:Filme)
RETURN node_pessoa, node_filme

2 rows affected.


node_pessoa,node_filme
"{'nome': 'Laura Lero', 'label': 'Laura'}","{'ano': 2009, 'titulo': 'Django', 'label': 'Django'}"
"{'nome': 'Matheus Mota', 'label': 'Matheus'}","{'ano': 2009, 'titulo': 'Django', 'label': 'Django'}"


In [13]:
%%cypher
MATCH (node_pessoa:Pessoa)-[:DEU_LIKE]->(node_filme:Filme)
WHERE  node_filme.titulo = 'Django'
RETURN node_pessoa.nome as nome_usuario, node_filme.titulo

2 rows affected.


nome_usuario,node_filme.titulo
Laura Lero,Django
Matheus Mota,Django
